# Digital Marketing Analytics - Group Assignment 1 

### Context

- Multi-channel company with sales of $700M per year
- Retail stores, Catalog, Website
- Brand is well known nationaly, strong positive reputation, good long term customer loyalty
- Food products, purchased as gifts for Christmas

### Marketing challenges

1. CRM (contact management across channels)
2. Competitive marketplace for Christmas gifts
3. Generating business off Christmas

### Dataset

4 files:
1. "DMEFExtractSummaryV01.csv": Customer file.
    - customer ID
    - Purchasing behaviour variables have a naming scheme with 3 components: 
        - Channel: Ret (retail), Int (internet) and C (catalog)
        - Season: F (fall) or S (spring) followed by the last 2 digits of the year. Summaries for the period prior to 2004 have the designation Pre04.
        - Measure: Dollars (Retail sales), GDollars (Gift), NGDollars (Non gift), Orders, Trips and Lines. For Pre04, Recency: recency of last purchase measured in number of season prior to 2004 (99 means not purchased in the channel prior to 2004)
    
    
    
    buying activity, channel summary, gift recipients counts by eason, catalog and email circulation counts by season for 6 seasons, first order characateristics, overlay demographic and psychographic data, estimated distance to retail store and sectional center code

2. "DMEFExtractContactsV01.csv": Marketing contact records, catalog and e-mail contacts
    customer ID, contact date and type of contact (2005-2007)
3. "DMEFExtractOrdersV01.csv": Order or store trip records 
    customer ID, order number, data, channel, payment method
4. "DMEFExtractLinesV01.csv": Line item records
    customer ID, order nulmber, date, price, gift flag, ID of gift recipients (2001-2007)

## Import libraries

In [18]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## EDA

In [33]:
customers = pd.read_csv('datasets/DMEFExtractSummaryV01.csv')
contacts = pd.read_csv('datasets/DMEFExtractContactsV01.csv')
orders = pd.read_csv('datasets/DMEFExtractOrdersV01.csv')
line_items = pd.read_csv('datasets/DMEFExtractLinesV01.csv')

3 differrent marketing channels: __retail, catalogue, website__

In [37]:
customers.columns

Index(['Cust_ID', 'SCF_Code', 'RetF07Dollars', 'RetF07Trips', 'RetF07Lines',
       'RetS07Dollars', 'RetS07Trips', 'RetS07Lines', 'RetF06Dollars',
       'RetF06Trips',
       ...
       'IncCode', 'HomeCode', 'Child0_2', 'Child3_5', 'Child6_11',
       'Child12_16', 'Child17_18', 'Dwelling', 'LengthRes', 'HomeValue'],
      dtype='object', length=170)

In [16]:
# Merge the dataframes
df = pd.merge(customers, contacts, on='customer_id')
df = pd.merge(df, orders, on='customer_id')
df = pd.merge(df, line_items, on='order_id')

KeyError: 'customer_id'

In [12]:
customers.columns()

TypeError: 'Index' object is not callable